In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import pandas as pd
from fredapi import Fred

In [16]:
start_date = '2000-01-01'
end_date = '2020-01-01'

# Fred API key: ab766afb0df13dba8492403a7865f852
fred = Fred(api_key = 'ab766afb0df13dba8492403a7865f852')

df11 = {}
df11['DTB4WK'] = fred.get_series('DTB4WK', observation_start=start_date, observation_end=end_date)
df11['DTB3'] = fred.get_series('DTB3', observation_start=start_date, observation_end=end_date)
df11['DTB6'] = fred.get_series('DTB6', observation_start=start_date, observation_end=end_date)
df11['DGS5'] = fred.get_series('DGS5', observation_start=start_date, observation_end=end_date)
df11['DGS10'] = fred.get_series('DGS10', observation_start=start_date, observation_end=end_date)
df11 = pd.DataFrame(df11)

In [10]:
T120 = fred.get_series('DGS10', observation_start=start_date, observation_end=end_date)
T1 = fred.get_series('DTB4WK', observation_start=start_date, observation_end=end_date)
T3 = fred.get_series('DTB3', observation_start=start_date, observation_end=end_date)
T6 = fred.get_series('DTB6', observation_start=start_date, observation_end=end_date)

BAA = fred.get_series('DBAA', observation_start=start_date, observation_end=end_date)
AAA = fred.get_series('DAAA', observation_start=start_date, observation_end=end_date)

In [11]:
def generate_feature(f1, f2):
    '''
    compare and choose latest start date and earliest end date
    '''
    start_date = max(f1.index[0], f2.index[0])
    end_date = min(f1.index[-1], f2.index[-1])
    feature = f1[(f1.index >= start_date) & (f1.index <= end_date)] \
        - f2[(f2.index >= start_date) & (f2.index <= end_date)]
    return feature

In [12]:
TE1 = generate_feature(T120, T1)
TE2 = generate_feature(T120, T3)
TE3 = generate_feature(T120, T6)
TE5 = generate_feature(T3, T1)
TE6 = generate_feature(T6, T1)
DE1 = generate_feature(BAA, AAA)
DE2 = generate_feature(BAA, T120)
DE4 = generate_feature(BAA, T6)
DE5 = generate_feature(BAA, T3)
DE6 = generate_feature(BAA, T1)

In [13]:
data = pd.read_csv("https://www.federalreserve.gov/datadownload/Output.aspx?rel=H15&series=bf17364827e38702b42a58cf8eaa3f78&lastobs=&from=&to=&filetype=csv&label=include&layout=seriescolumn&type=package")
data = data.iloc[5:,:]
data['Series Description'] = pd.to_datetime(data['Series Description'])
CTB3M = pd.Series(data['Market yield on U.S. Treasury securities at 3-month   constant maturity, quoted on investment basis'])
CTB3M.reset_index(drop=True, inplace=True)
CTB3M.index = data['Series Description'].values
CTB3M = CTB3M.dropna()
CTB3M = CTB3M[CTB3M.values != 'ND'].astype(float)
CTB3M = CTB3M[(CTB3M.index >= start_date) & (CTB3M.index <= end_date)]
CTB3M = CTB3M - CTB3M.shift()

In [14]:
CTB6M = pd.Series(data['Market yield on U.S. Treasury securities at 6-month   constant maturity, quoted on investment basis'])
CTB6M.reset_index(drop=True, inplace=True)
CTB6M.index = data['Series Description'].values
CTB6M = CTB6M.dropna()
CTB6M = CTB6M[CTB6M.values != 'ND'].astype(float)
CTB6M = CTB6M[(CTB6M.index >= start_date) & (CTB6M.index <= end_date)]
CTB6M = CTB6M - CTB6M.shift()
CTB6M

2000-01-03     NaN
2000-01-04   -0.06
2000-01-05   -0.01
2000-01-06   -0.05
2000-01-07   -0.03
              ... 
2019-12-24    0.01
2019-12-26    0.00
2019-12-27   -0.02
2019-12-30    0.01
2019-12-31    0.00
Name: Market yield on U.S. Treasury securities at 6-month   constant maturity, quoted on investment basis, Length: 5002, dtype: float64

In [15]:
CTB1Y = pd.Series(data['Market yield on U.S. Treasury securities at 1-year   constant maturity, quoted on investment basis'])
CTB1Y.reset_index(drop=True, inplace=True)
CTB1Y.index = data['Series Description'].values
CTB1Y = CTB1Y.dropna()
CTB1Y = CTB1Y[CTB1Y.values != 'ND'].astype(float)
CTB1Y = CTB1Y[(CTB1Y.index >= start_date) & (CTB1Y.index <= end_date)]
CTB1Y = CTB1Y - CTB1Y.shift()
CTB1Y

2000-01-03     NaN
2000-01-04   -0.09
2000-01-05    0.05
2000-01-06   -0.02
2000-01-07   -0.03
              ... 
2019-12-24    0.00
2019-12-26    0.00
2019-12-27   -0.02
2019-12-30    0.06
2019-12-31    0.02
Name: Market yield on U.S. Treasury securities at 1-year   constant maturity, quoted on investment basis, Length: 5002, dtype: float64

In [ ]:
# 2 features not implemented yet for Paper 7

#Oil - Relative change of oil price (WTI), Oklahoma
#Gold - Relative change of gold price (London market)